In [1]:
import numpy as np
import warnings
from tqdm import tqdm

import torch
from torch.utils.data import DataLoader

import pytorch_lightning as pl

from deeptime.models.multiclass.resnet import ResNetClassifier
from deeptime.data import BaseDataset

from sktime.datasets import load_UCR_UEA_dataset

C:\Users\medei\AppData\Local\Programs\Python\Python39\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
warnings.filterwarnings('ignore')

In [3]:
DATASETS = ['ECG5000', 'Haptics', 'Worms']
ACTIVATION = 'relu'
NUM_CLASSES = 5

In [4]:
class LitProgressBar(pl.callbacks.TQDMProgressBar):

    def init_validation_tqdm(self):
        bar = tqdm(
            disable=True,
        )
        return bar

In [5]:
for dataset in DATASETS:
    print()
    print('-'*50)
    print(f'Executing experiment with dataset {dataset}')
    accs = []

    for _ in range(5):
        x_train, y_train = load_UCR_UEA_dataset(name=dataset, split='train')
        y_train = np.array(y_train, dtype=np.int32)
        y_train = y_train - 1
        y_train = np.eye(NUM_CLASSES)[y_train]

        x_test, y_test = load_UCR_UEA_dataset(name=dataset, split='test')
        y_test = np.array(y_test, dtype='int32') # Fixing the labels type
        y_test = y_test - 1
        y_test = np.eye(NUM_CLASSES)[y_test]

        x_train_transformed = []
        for val in x_train.values:
            x_train_transformed.append(val[0].tolist())
        x_train = np.array(x_train_transformed)

        x_test_transformed = []
        for val in x_test.values:
            x_test_transformed.append(val[0].tolist())
        x_test = np.array(x_test_transformed)

        x_train = np.expand_dims(x_train_transformed, axis=1)
        x_test = np.expand_dims(x_test_transformed, axis=1)

        full_train_dataset = BaseDataset(x=x_train, y=y_train)
        train_loader = DataLoader(full_train_dataset, batch_size=32)

        test_dataset = BaseDataset(x=x_test, y=y_test)
        test_loader = DataLoader(test_dataset, batch_size=32)

        model = ResNetClassifier(
            in_channels=1,
            num_classes=NUM_CLASSES,
            activation=ACTIVATION
        )
    
        trainer = pl.Trainer(
            max_epochs=100,
            accelerator='gpu',
            devices=-1,
            callbacks=[LitProgressBar()]
        )

        trainer.fit(model, train_dataloaders=train_loader)
        results = trainer.test(model, test_loader)[0]
        accs.append(results['Accuracy Score'])

    print(f"Accuracy: {accs}")


--------------------------------------------------
Executing experiment with dataset ECG5000


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type             | Params
-----------------------------------------------
0 | layers    | Sequential       | 126 K 
1 | final     | Sequential       | 325   
2 | criterion | CrossEntropyLoss | 0     
-----------------------------------------------
127 K     Trainable params
0         Non-trainable params
127 K     Total params
0.509     Total estimated model params size (MB)


Epoch 99: 100%|█████████████████████████████████| 16/16 [00:00<00:00, 28.51it/s, loss=1.04, v_num=26, train_loss=0.974]

`Trainer.fit` stopped: `max_epochs=100` reached.


Epoch 99: 100%|█████████████████████████████████| 16/16 [00:00<00:00, 25.94it/s, loss=1.04, v_num=26, train_loss=0.974]

...

Epoch 99: 100%|███████████████████████████████████| 6/6 [00:00<00:00, 27.02it/s, loss=1.17, v_num=40, train_loss=1.150]

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]



Testing DataLoader 0: 100%|██████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 75.46it/s]


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│      Accuracy Score       │    0.4935064935064935     │
└───────────────────────────┴───────────────────────────┘

Accuracy: [0.5974025974025974, 0.4155844155844156, 0.5324675324675324, 0.5844155844155844, 0.4935064935064935]


In [7]:
import numpy as np

results_ecg5000 = np.array([0.9271111111111111, 0.5493333333333333, 0.7402222222222222, 0.7388888888888889, 0.8975555555555556])
results_haptics = np.array([0.4253246753246753, 0.3344155844155844, 0.4090909090909091, 0.36038961038961037, 0.4448051948051948])
results_worms = np.array([0.5974025974025974, 0.4155844155844156, 0.5324675324675324, 0.5844155844155844, 0.4935064935064935])

print(results_ecg5000.mean(), results_ecg5000.std())
print(results_haptics.mean(), results_haptics.std())
print(results_worms.mean(), results_worms.std())

0.7706222222222222 0.13527806935857234
0.39480519480519477 0.04115059615001264
0.5246753246753246 0.06601696117994198


In [5]:
ACTIVATION = 'lisin'

In [6]:
for dataset in DATASETS:
    print()
    print('-'*50)
    print(f'Executing experiment with dataset {dataset}')
    accs = []

    for _ in range(5):
        x_train, y_train = load_UCR_UEA_dataset(name=dataset, split='train')
        y_train = np.array(y_train, dtype=np.int32)
        y_train = y_train - 1
        y_train = np.eye(NUM_CLASSES)[y_train]

        x_test, y_test = load_UCR_UEA_dataset(name=dataset, split='test')
        y_test = np.array(y_test, dtype='int32') # Fixing the labels type
        y_test = y_test - 1
        y_test = np.eye(NUM_CLASSES)[y_test]

        x_train_transformed = []
        for val in x_train.values:
            x_train_transformed.append(val[0].tolist())
        x_train = np.array(x_train_transformed)

        x_test_transformed = []
        for val in x_test.values:
            x_test_transformed.append(val[0].tolist())
        x_test = np.array(x_test_transformed)

        x_train = np.expand_dims(x_train_transformed, axis=1)
        x_test = np.expand_dims(x_test_transformed, axis=1)

        full_train_dataset = BaseDataset(x=x_train, y=y_train)
        train_loader = DataLoader(full_train_dataset, batch_size=32)

        test_dataset = BaseDataset(x=x_test, y=y_test)
        test_loader = DataLoader(test_dataset, batch_size=32)

        model = ResNetClassifier(
            in_channels=1,
            num_classes=NUM_CLASSES,
            activation=ACTIVATION
        )

        trainer = pl.Trainer(
            max_epochs=100,
            accelerator='gpu',
            devices=-1,
            callbacks=[LitProgressBar()]
        )

        trainer.fit(model, train_dataloaders=train_loader)
        results = trainer.test(model, test_loader)[0]
        accs.append(results['Accuracy Score'])

    print(f"Accuracy: {accs}")


--------------------------------------------------
Executing experiment with dataset ECG5000


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type             | Params
-----------------------------------------------
0 | layers    | Sequential       | 126 K 
1 | final     | Sequential       | 325   
2 | criterion | CrossEntropyLoss | 0     
-----------------------------------------------
127 K     Trainable params
0         Non-trainable params
127 K     Total params
0.509     Total estimated model params size (MB)


Epoch 99: 100%|█████████████████████████████████| 16/16 [00:00<00:00, 37.30it/s, loss=1.01, v_num=41, train_loss=0.957]

`Trainer.fit` stopped: `max_epochs=100` reached.


Epoch 99: 100%|█████████████████████████████████| 16/16 [00:00<00:00, 35.20it/s, loss=1.01, v_num=41, train_loss=0.957]

...

Epoch 99: 100%|███████████████████████████████████| 6/6 [00:00<00:00, 21.91it/s, loss=1.22, v_num=55, train_loss=1.200]


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing DataLoader 0: 100%|██████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 45.89it/s]


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│      Accuracy Score       │    0.2077922077922078     │
└───────────────────────────┴───────────────────────────┘

Accuracy: [0.4805194805194805, 0.44155844155844154, 0.4675324675324675, 0.4805194805194805, 0.2077922077922078]


In [7]:
import numpy as np

results_ecg5000 = np.array([0.9248888888888889, 0.8553333333333333, 0.9284444444444444, 0.9224444444444444, 0.8951111111111111])
results_haptics = np.array([0.22727272727272727, 0.24675324675324675, 0.25, 0.24675324675324675, 0.2662337662337662])
results_worms = np.array([0.4805194805194805, 0.44155844155844154, 0.4675324675324675, 0.4805194805194805, 0.2077922077922078])

print(results_ecg5000.mean(), results_ecg5000.std())
print(results_haptics.mean(), results_haptics.std())
print(results_worms.mean(), results_worms.std())

0.9052444444444445 0.027618065299079113
0.24740259740259737 0.012388820797622666
0.41558441558441556 0.10486560643213687


In [10]:
ACTIVATION = 'lisilu'

for dataset in DATASETS:
    print()
    print('-'*50)
    print(f'Executing experiment with dataset {dataset}')
    accs = []

    for _ in range(5):
        x_train, y_train = load_UCR_UEA_dataset(name=dataset, split='train')
        y_train = np.array(y_train, dtype=np.int32)
        y_train = y_train - 1
        y_train = np.eye(NUM_CLASSES)[y_train]

        x_test, y_test = load_UCR_UEA_dataset(name=dataset, split='test')
        y_test = np.array(y_test, dtype='int32') # Fixing the labels type
        y_test = y_test - 1
        y_test = np.eye(NUM_CLASSES)[y_test]

        x_train_transformed = []
        for val in x_train.values:
            x_train_transformed.append(val[0].tolist())
        x_train = np.array(x_train_transformed)

        x_test_transformed = []
        for val in x_test.values:
            x_test_transformed.append(val[0].tolist())
        x_test = np.array(x_test_transformed)

        x_train = np.expand_dims(x_train_transformed, axis=1)
        x_test = np.expand_dims(x_test_transformed, axis=1)

        full_train_dataset = BaseDataset(x=x_train, y=y_train)
        train_loader = DataLoader(full_train_dataset, batch_size=32)

        test_dataset = BaseDataset(x=x_test, y=y_test)
        test_loader = DataLoader(test_dataset, batch_size=32)

        model = ResNetClassifier(
            in_channels=1,
            num_classes=NUM_CLASSES,
            activation=ACTIVATION
        )

        trainer = pl.Trainer(
            max_epochs=100,
            accelerator='gpu',
            devices=-1,
            callbacks=[LitProgressBar()]
        )

        trainer.fit(model, train_dataloaders=train_loader)
        results = trainer.test(model, test_loader)[0]
        accs.append(results['Accuracy Score'])

    print(f"Accuracy: {accs}")


--------------------------------------------------
Executing experiment with dataset ECG5000


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type             | Params
-----------------------------------------------
0 | layers    | Sequential       | 126 K 
1 | final     | Sequential       | 325   
2 | criterion | CrossEntropyLoss | 0     
-----------------------------------------------
127 K     Trainable params
0         Non-trainable params
127 K     Total params
0.509     Total estimated model params size (MB)


Epoch 99: 100%|█████████████████████████████████| 16/16 [00:00<00:00, 32.38it/s, loss=1.04, v_num=56, train_loss=0.968]

`Trainer.fit` stopped: `max_epochs=100` reached.


Epoch 99: 100%|█████████████████████████████████| 16/16 [00:00<00:00, 30.81it/s, loss=1.04, v_num=56, train_loss=0.968]

...

Epoch 15:  20%|███████                            | 1/5 [00:00<00:00, 28.48it/s, loss=1.49, v_num=64, train_loss=1.480]

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing DataLoader 0:  90%|██████████████████████████████████████████████████████▉      | 9/10 [00:00<00:00, 77.82it/s]

TypeError: 'NoneType' object is not subscriptable

Epoch 3: 100%|████████████████████████████████████| 5/5 [00:19<00:00,  3.94s/it, loss=1.57, v_num=63, train_loss=1.560]

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type             | Params
-----------------------------------------------
0 | layers    | Sequential       | 126 K 
1 | final     | Sequential       | 325   
2 | criterion | CrossEntropyLoss | 0     
-----------------------------------------------
127 K     Trainable params
0         Non-trainable params
127 K     Total params
0.509     Total estimated model params size (MB)


Epoch 99: 100%|█████████████████████████████████| 16/16 [00:00<00:00, 33.43it/s, loss=1.03, v_num=19, train_loss=0.966]

`Trainer.fit` stopped: `max_epochs=100` reached.


Epoch 99: 100%|█████████████████████████████████| 16/16 [00:00<00:00, 31.72it/s, loss=1.03, v_num=19, train_loss=0.966]


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing DataLoader 0: 100%|█████████████████████████████████████████████████████████| 141/141 [00:01<00:00, 109.92it/s]


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│      Accuracy Score       │    0.5995555555555555     │
└───────────────────────────┴───────────────────────────┘

[{'Accuracy Score': 0.5995555555555555}]